In [1]:
import pandas as pd
adult_census = pd.read_csv("../datasets/adult-census.csv")
adult_census = adult_census.drop(columns='education-num')

target_name = 'class'
target = adult_census[target_name]
data = adult_census.drop(columns = [target_name])

In [4]:
from sklearn.compose import make_column_selector as selector

categorical_selector = selector(dtype_include=object)
numerical_selector = selector(dtype_exclude=object)

numerical_columns = numerical_selector(data)
categorical_columns = categorical_selector(data)

print('numerical features', numerical_columns)
print('categorical features', categorical_columns)

numerical features ['age', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical features ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


In [5]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [6]:
categorical_preprocessor = OneHotEncoder(handle_unknown='ignore')
numerical_preprocessor = StandardScaler()



In [7]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ('onehot_encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)
])

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

model = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=500))

model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['age', 'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [9]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.75, random_state=22)

In [10]:
_ = model.fit(data_train, target_train)

In [11]:
pred = model.predict(data_test)
pred[:10]

array([' <=50K', ' >50K', ' >50K', ' <=50K', ' <=50K', ' <=50K', ' <=50K',
       ' <=50K', ' <=50K', ' >50K'], dtype=object)

In [14]:
target_test[:20] == pred[:20]

35057     True
7449      True
22289     True
11354     True
30394     True
41467     True
43462     True
10231     True
14557     True
19433     True
26327    False
9704      True
43714     True
16733    False
2628      True
13978     True
3970      True
18821     True
34141     True
42573     True
Name: class, dtype: bool

In [15]:
(pred == target_test).mean()

0.8561133404307592